In [ ]:
import pandas as pd
from pathlib import Path
from rhscripts import dcm
from pydicom import dcmread
import shutil

In [ ]:
# Data paths
path_RTSS = Path('../data/RTSS')
path_data = Path('../data/')

In [ ]:
# Keep these folders, delete all others of same modality
resolve = {}
resolve['Cmet0216'] = '1.2.124.113532.1641892232101216572131160211235205112142116'
resolve['Cmet0392'] = '1.3.12.2.1107.5.1.4.11086.30000019051507323768900025349'
resolve['Cmet1354'] = '1.2.840.113619.2.290.3.397169200.368.1550473194.624.13'

In [ ]:
delete_folders = []
do_delete=True

for pt in path_data.glob('controls/*'):
    if len(list(pt.iterdir())) == 2: 
        continue

    delete_folders.append(pt)

for pt in path_data.glob('anonymized/*'):
    if len(list(pt.iterdir())) == 2: 
        continue
    
    rtss_match = len(RTSS_file := list(path_RTSS.glob(f'{pt.name}/*/*')))

    if rtss_match == 0:
        delete_folders.append(pt)
    if rtss_match == 1:
        print(pt.name, len(list(pt.iterdir())))
        reference_suid = dcm.get_reference_seriesUID_from_RTSS(RTSS_file[0])

        # CHECK CANDIDATE DICOMS
        candidates = {'PT': {}, 'CT': {}}
        container = None
        container_modality = None
        for dcm_dir in pt.iterdir():
            ds = dcmread(next(dcm_dir.iterdir()))
            candidates[ds.Modality][dcm_dir.absolute()] = ds.SeriesDescription
            if ds.SeriesInstanceUID == reference_suid:
                container = dcm_dir.absolute()
                container_modality = ds.Modality
        for modality in ['PT','CT']:
            del_count=0
            if len(candidates[modality])==1:
                pass # All is well
            elif container_modality == modality:
                # Delete all that is not container
                for k in candidates[modality].keys():
                    if not k == container:
                        delete_folders.append(k)
                        del_count+=1
            elif pt.name in resolve:
                for k in candidates[modality].keys():
                    if not Path(k).name == resolve[pt.name]:
                        delete_folders.append(k)
                        del_count+=1
            else:
                print(candidates[modality])
                do_delete=False
            
            if not del_count == len(candidates[modality])-1:
                print("\t",modality,"PROBLEMS...",del_count)
                do_delete=False
    
for d in delete_folders:
    print("Deleting",d)
    if do_delete:
        shutil.rmtree(d)

In [ ]:
len(delete_folders)